In [7]:
#Copyright (c) 2019 Primoz Ravbar UCSB Licensed under BSD 2-Clause [see LICENSE for details] Written by Primoz Ravbar

##ABRS Label Maker GUI 
# Originally written by Primoz Ravbar
# Edited by Venkata Manikanta Muriki

##################
### START CODE ###
##################
import warnings
warnings.filterwarnings('ignore')

from tkinter import * # GUI toolkit
from tkinter import filedialog
from tkinter import messagebox
from tkinter import ttk
from tkinter import font
import matplotlib
import matplotlib.pyplot as plt
import math
import datetime

from matplotlib.backends.backend_tkagg import (
    FigureCanvasTkAgg, NavigationToolbar2Tk)
# Implement the default Matplotlib key bindings.
from matplotlib.backend_bases import key_press_handler
from matplotlib.figure import Figure
import mplcursors

import cv2
import PIL.Image, PIL.ImageTk
from PIL import Image
import time
import numpy as np
import pickle

from ABRS_modules import *

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential

import os


root = Tk()

class LabelMakerGUI():
    def __init__(self, master, dictParam): 
        '''
        WRITE
        '''
        self.master = master
        self.fullscreen_mode()
        self.master.resizable(0,0)
        self.master.title('ABRS Label Maker GUI')
        self.create_menu()
        
              
        self.boolDoRun = False 
        
        # Instantiating dictParam containing video_source, ABRS_model_path, etc.       
        self.dictParam = dictParam
         
        self.video_source = self.dictParam['video_source']
        
        self.ABRS_model_path = self.dictParam['ABRS_model_path']
            
        self.quadrant = self.dictParam['quadrant']
        self.quadrantDict = {1 : "Top Left", 2 : "Top Right", 3 : "Bottom Left", 4 : "Bottom Right", 0 : "All", 5 : "None"}
    
        self.quadrant_name = self.quadrantDict[self.quadrant]
        
        self.ethoLength = self.dictParam['zoomEthoLength']  

        self.project_name = self.dictParam['project_name']
        self.alpha = self.dictParam['alpha']
        self.first_time_bool = self.dictParam['first_time_bool'] # for ABRS cust.dat
            
        self.startFrame = 0
        self.endFrame = 25
        self.OldLabel = 0
        self.zoomIndStart = 0
        self.zoomIndEnd = self.endFrame - self.startFrame + 1
        self.FirstRunBool = True
        self.draw_etho_first = True  # the first ethogram drawn, the others will be for revised ethogramss
        self.set_selector_lines_back_bool = False
        self.set_first_selector_line_back = False
        self.set_both_selector_lines_back = False
        self.load_all_framesBool = False
        self.ZoomHome = False
        self.Zoom1x = False
        self.Zoom2x = False
        self.in_xi_FirstSelector = 0
        self.in_xi_SecondSelector = 0
        self.two_label_inputs = False  # need this for secondary label 
        self.call_self_photo_bind = True # need this for tag bind
        
        self.frame_count = 0
        self.duration = 0
        self.fps = 0
        self.raw_frame_zoomout = True
        
        self.correctingBool = False
        
        self.create_layout()
    
        self.create_start_page()
        
        self.master.mainloop()
    
    def fullscreen_mode(self):
        self.master.attributes('-topmost', True)
        self.master.attributes('-fullscreen', True)
    
    def create_start_page(self):
        self.master.attributes('-topmost', False)
        self.StartPage = Toplevel()  # Top object
        self.StartPage.title('Start Page')
        self.StartPage.configure(bg = 'honeydew2')
        self.StartPage.attributes('-topmost', True)
        
        self.welcome_label = Label(self.StartPage, text = "Welcome to ABRS Label Maker GUI", bg = "skyblue")
        self.welcome_label.pack()
        
        self.project_video_canvas = Canvas(self.StartPage, bg = 'honeydew2', bd = 6)
        self.project_video_canvas.pack(side = LEFT, fill = Y)
        
        self.spacefiller_vid_canvL = Label(self.project_video_canvas, bg = 'honeydew2')   
        self.spacefiller_vid_canvL.pack(side = LEFT, fill = Y)
        
        self.spacefiller_vid_canvR = Label(self.project_video_canvas, bg = 'honeydew2')   
        self.spacefiller_vid_canvR.pack(side = RIGHT, fill = Y)
        
        self.spacefiller_vid_canvT = Label(self.project_video_canvas, bg = 'honeydew2')   
        self.spacefiller_vid_canvT.pack(fill = X)
        
        self.project_entry = Entry(self.project_video_canvas)
        self.project_entry.insert(END, self.project_name)
        self.project_entry.pack()
        
        
        self.spacefiller1 = Label(self.project_video_canvas, bg = 'honeydew2')   
        self.spacefiller1.pack()
        
        self.spacefiller1_5 = Label(self.project_video_canvas, text = "Current Video:", bg = 'honeydew2')   
        self.spacefiller1_5.pack()

        
        self.video_name_display = Label(self.project_video_canvas, bg = 'honeydew2', text = str(self.video_source))
        self.video_name_display.pack()
        
        def select_video_file():
            self.StartPage.attributes('-topmost', False)
            self.video_input_file = filedialog.askopenfilename(title="Open File", initialdir=(os.path.expanduser('~/')))
            self.StartPage.attributes('-topmost', True)
            if not self.video_input_file:
                return None
            self.video_string = self.video_input_file.replace("/", "//")
            self.video_name_display.configure(text = self.video_string)
            
        self.change_videofile = ttk.Button(self.project_video_canvas, text = "Change Video", command = select_video_file)   
        self.change_videofile.pack()
        
        self.spacefiller2 = Label(self.project_video_canvas, bg = 'honeydew2')   
        self.spacefiller2.pack()
        
        self.spacefiller2_5 = Label(self.project_video_canvas, text = "Current Model:", bg = 'honeydew2')   
        self.spacefiller2_5.pack()
        
        
        self.model_name_display = Label(self.project_video_canvas, bg = 'honeydew2', text = str(self.ABRS_model_path))
        self.model_name_display.pack(fill = X)
        
        def select_model_file():
            self.StartPage.attributes('-topmost', False)
            self.model_input_file = filedialog.askopenfilename()
        
            self.StartPage.attributes('-topmost', True)
            if not self.model_input_file:
                return None
            self.model_string = self.model_input_file.replace("/", "//")
            self.model_name_display.configure(text = self.model_string)

        
        self.change_modelfile = ttk.Button(self.project_video_canvas, text = "Change Model", command = select_model_file)   
        self.change_modelfile.pack()
        
        self.spacefiller3 = Label(self.project_video_canvas, bg = 'honeydew2')   
        self.spacefiller3.pack()
                
        self.r = IntVar()
        self.r.set(str(self.quadrant))
        
        self.radioGroup = LabelFrame(self.project_video_canvas, text = "Select Quadrant", bg = 'honeydew2')
        self.radioGroup.pack()
        
        Radiobutton(self.radioGroup, text = 'Top Left', variable = self.r, value = 1, bg = 'honeydew2').grid(row = 0, column = 0, sticky = W)
        Radiobutton(self.radioGroup, text = 'Top Right', variable = self.r, value = 2, bg = 'honeydew2').grid(row = 0, column = 1, sticky = E)
        Radiobutton(self.radioGroup, text = 'Bottom Left', variable = self.r, value = 3, bg = 'honeydew2').grid(row = 1, column = 0, sticky = W)
        Radiobutton(self.radioGroup, text = 'Bottom Right', variable = self.r, value = 4, bg = 'honeydew2').grid(row = 1, column = 1, sticky = E)
        Radiobutton(self.radioGroup, text = 'All', variable = self.r, value = 0, bg = 'honeydew2').grid(row = 2, column = 0, sticky = EW)

        self.spacefillerr = Label(self.project_video_canvas, bg = 'honeydew2')   
        self.spacefillerr.pack()
             
        
        def listbox_delete():
            if self.my_listbox.curselection():
                self.my_listbox.delete(self.my_listbox.curselection()[0], self.my_listbox.curselection()[-1])
            
        def listbox_add():
            self.my_listbox.insert(END, "[" + str(self.my_listbox.size()+1) + "] " + self.lb_add_entry.get())
            self.my_listbox.select_set(self.my_listbox.size())
            self.lb_add_entry.delete(0, END)
                
        self.spacefillerr_Rtop = Label(self.StartPage, bg = 'honeydew2')   
        self.spacefillerr_Rtop.pack()

        self.listbox_del_btn = ttk.Button(self.StartPage, text = "Delete", command = listbox_delete)
        self.listbox_del_btn.pack()
        
        self.my_listbox = Listbox(self.StartPage, selectmode = EXTENDED)
        self.my_listbox.pack()
        
        for self.item in self.alpha:
            self.my_listbox.insert(END, self.item)
        
        self.spacefillerr_Rend23 = Label(self.StartPage, bg = 'honeydew2')   
        self.spacefillerr_Rend23.pack()
        
        self.lb_add_entry = Entry(self.StartPage)
        self.lb_add_entry.pack()
        
        self.lb_add_btn = ttk.Button(self.StartPage, text = "Add", command = listbox_add)
        self.lb_add_btn.pack()
        
        self.spacefillerr_Rend = Label(self.StartPage, bg = 'honeydew2')   
        self.spacefillerr_Rend.pack()
        
        self.spacefillerr_Rend2 = Label(self.StartPage, bg = 'honeydew2')   
        self.spacefillerr_Rend2.pack()
        
        self.check_var = IntVar()
        self.check_var.set(0)
        self.set_default = ttk.Checkbutton(self.StartPage, text = "Set as default", variable = self.check_var)
        if self.first_time_bool == False:
            self.set_default.pack()
        
        def finalize_start_selections():
            self.StartPage.attributes('-topmost', False)
            self.start_save = messagebox.askyesno('', 'Are you sure you want to continue with all selections made above?', parent = self.StartPage)

            if self.start_save == 1:
                
                self.video_source = self.video_name_display.cget('text')
                self.ABRS_model_path = self.model_name_display.cget('text')
                self.alpha = list(self.my_listbox.get(0, END)) 
                self.project_name = self.project_entry.get()
                self.quadrant = self.r.get() 
                self.quadrant_name = self.quadrantDict[self.quadrant]
                
                self.dictParam = {"zoomEthoLength" : 100, "ABRS_model_path" : self.ABRS_model_path,
                                  "video_source" : self.video_source,
                                  "quadrant" : self.quadrant, "project_name" : self.project_name, "alpha" : self.alpha,
                                  "first_time_bool" : False}
                    
                cust_ok, warn_message = self.check_customization()
                if cust_ok:
                    self.video_source = self.video_name_display.cget('text')
                    self.ABRS_model_path = self.model_name_display.cget('text')
                    self.alpha = list(self.my_listbox.get(0, END)) # tuple to list
                    self.project_name = self.project_entry.get()
                    self.quadrant = self.r.get() # Change this to All after this project is done because this project has multuple windows
                    self.quadrant_name = self.quadrantDict[self.quadrant]
                    self.HistoryLabel = Label(self.Historysecond_frame, text = datetime.datetime.now().strftime(("%H:%M:%S")) + " \"" + str(self.project_name) + "\"" + ' opened', justify = LEFT)
                    self.HistoryLabel.grid(row = self.history_counter, column = 0, sticky = W)
                    self.increase_history_counter()
                    self.master.update_idletasks()
                    self.HistoryCanvas2.configure(scrollregion=self.HistoryCanvas2.bbox("all")) 
                    self.HistoryCanvas2.yview_moveto(1)

                    self.calculate_video_info()
                    
                    self.new_lb.delete(0)
                    self.new_lb.insert(0, str(self.video_source))
                    self.new_lb.delete(1)
                    self.new_lb.insert(1, str(self.frame_count))
                    self.new_lb.delete(2)
                    self.new_lb.insert(2, str(self.duration) + ' sec')
                    self.new_lb.delete(3)
                    self.new_lb.insert(3, str(self.fps))
                    self.new_lb.delete(4)
                    self.new_lb.insert(4, str(self.quadrant_name))
                    self.new_lb.delete(5)
                    self.new_lb.insert(5, str(self.ABRS_model_path))
                                        
                    if self.check_var.get() == 1:
                        pickle.dump(self.dictParam, open("ABRS_cust.dat", "wb"))
                        self.HistoryLabel = Label(self.Historysecond_frame, text = datetime.datetime.now().strftime(("%H:%M:%S")) + ' New Default Options Saved\nto File "ABRS_cust.dat"', justify = LEFT)
                        self.HistoryLabel.grid(row = self.history_counter, column = 0, sticky = W)
                        self.increase_history_counter()
                        self.master.update_idletasks()
                        self.HistoryCanvas2.configure(scrollregion=self.HistoryCanvas2.bbox("all")) 
                        self.HistoryCanvas2.yview_moveto(1)

                    self.StartPage.destroy()
                else:
            
                    messagebox.showerror('', warn_message, parent = self.StartPage)
                    
            else:
                pass
                self.StartPage.attributes('-topmost', True)
       
        self.spacefillerr_Rend2_new = Label(self.StartPage, bg = 'honeydew2')   
        self.spacefillerr_Rend2_new.pack()
        
        self.finalize_btn = ttk.Button(self.StartPage, text = "Finalize Selections", command = finalize_start_selections)
        self.finalize_btn.pack()
        
        self.spacefiller45_new = Label(self.StartPage, bg = 'honeydew2')   
        self.spacefiller45_new.pack()
        
    
    def create_menu(self):
        main_menu = Menu(self.master) 
        self.master.config(menu = main_menu)
        file_menu = Menu(main_menu)
        main_menu.add_cascade(label = 'File', menu = file_menu)
        file_menu.add_command(label = 'Minimize', command = self.minimize_window)
        file_menu.add_command(label = 'Maximize', command = self.maximize_window)
        file_menu.add_command(label = 'Save and Finish', command = self.save_labels_and_images)
        file_menu.add_separator() # Adds thick line between two file_menu commands
        file_menu.add_command(label = 'Quit', command = self.system_quit)   
            
    def system_quit(self):
        if Toplevel.winfo_exists(self.StartPage) == True:
            self.master.attributes('-topmost', False)
            self.StartPage.attributes('-topmost', False)
        else:
            self.master.attributes('-topmost', False)
        response_quit = messagebox.askyesno('', 'Are you sure you want to quit? Your work is not saved.', parent = self.master)
        
        if response_quit == 1:
            self.master.quit()
            self.master.destroy()
        else:
            pass
            if Toplevel.winfo_exists(self.StartPage) == True:
                self.master.attributes('-topmost', False)
                self.StartPage.attributes('-topmost', False)
            else:
                self.master.attributes('-topmost', False)


    def create_layout(self):
        self.master.update_idletasks() 
        self.detect_width = self.master.winfo_screenwidth()
        self.detect_height = self.master.winfo_screenheight()
        
        if self.detect_width == 1920 and self.detect_height == 1200:  #756
            self.video_feed_dim = 756
            # Middle canvas
            self.frame_selector_canvas = Canvas(self.master, bg = 'honeydew2')
            self.frame_selector_canvas.place(x = 0, rely = 0.63, relwidth = 1, relheight = 0.07)
            
            # Ethogram canvas
            self.etho_canvas = Canvas(self.master, bg = 'bisque', bd = 0) 
            self.etho_canvas.place(relx = 0, rely = 0.7, relwidth = 1, relheight = 0.3)

            # Color image canvas
            self.color_canvas = Canvas(self.master, bg = 'gray25')
            self.color_canvas.place(relx = 0, rely = 0, relwidth = 0.39375, relheight = 0.63)  # change to ratio, this works for 1440x900 resolution 
            
            # ST image canvas
            self.st_canvas = Canvas(self.master, bg = 'gray25')
            self.st_canvas.place(relx = 0.39375, rely = 0, relwidth = 0.39375, relheight = 0.63)
            
            # frame loading canvas
            self.frame_loading_canvas = Canvas(self.master, bg = 'honeydew2')
            self.frame_loading_canvas.place(relx = 0.7875, rely = 0, relwidth = 0.2125, relheight = 0.63)
        
        
        if self.detect_width == 1440 and self.detect_height == 900:  #540
            # Middle canvas
            self.video_feed_dim = 540
            self.frame_selector_canvas = Canvas(self.master, bg = 'honeydew2')
            self.frame_selector_canvas.place(x = 0, rely = 0.6, relwidth = 1, relheight = 0.08)
            
            # Ethogram canvas
            self.etho_canvas = Canvas(self.master, bg = 'bisque', bd = 0) 
            self.etho_canvas.place(relx = 0, rely = 0.68, relwidth = 1, relheight = 0.32)

            # Color image canvas
            self.color_canvas = Canvas(self.master, bg = 'gray25')
            self.color_canvas.place(relx = 0, rely = 0, relwidth = 0.375, relheight = 0.6)  # change to ratio, this works for 1440x900 resolution 
            
            # ST image canvas
            self.st_canvas = Canvas(self.master, bg = 'gray25')
            self.st_canvas.place(relx = 0.375, rely = 0, relwidth = 0.375, relheight = 0.6)
            
            # frame loading canvas
            self.frame_loading_canvas = Canvas(self.master, bg = 'honeydew2')
            self.frame_loading_canvas.place(relx = 0.75, rely = 0, relwidth = 0.25, relheight = 0.6)
        
        if self.detect_width == 1366 and self.detect_height == 768: # 480 as mediator magic number
            # Middle canvas
            self.video_feed_dim = 480
            self.frame_selector_canvas = Canvas(self.master, bg = 'honeydew2')
            self.frame_selector_canvas.place(x = 0, rely = 0.625, relwidth = 1, relheight = 0.07)
            
            # Ethogram canvas
            self.etho_canvas = Canvas(self.master, bg = 'bisque', bd = 0) 
            self.etho_canvas.place(relx = 0, rely = 0.695, relwidth = 1, relheight = 0.305)

            # Color image canvas
            self.color_canvas = Canvas(self.master, bg = 'gray25')
            self.color_canvas.place(relx = 0, rely = 0, relwidth = 0.3514, relheight = 0.625)  # change to ratio, this works for 1440x900 resolution 
            
            # ST image canvas
            self.st_canvas = Canvas(self.master, bg = 'gray25')
            self.st_canvas.place(relx = 0.3514, rely = 0, relwidth = 0.3514, relheight = 0.625)
            
            # frame loading canvas
            self.frame_loading_canvas = Canvas(self.master, bg = 'honeydew2')
            self.frame_loading_canvas.place(relx = 0.7028, rely = 0, relwidth = 0.2972, relheight = 0.625)
            
        if self.detect_width == 1920 and self.detect_height == 1080:  #675
            self.video_feed_dim = 675
            # Middle canvas
            self.frame_selector_canvas = Canvas(self.master, bg = 'honeydew2')
            self.frame_selector_canvas.place(x = 0, rely = 0.625, relwidth = 1, relheight = 0.07)
            
            # Ethogram canvas
            self.etho_canvas = Canvas(self.master, bg = 'bisque', bd = 0) 
            self.etho_canvas.place(relx = 0, rely = 0.695, relwidth = 1, relheight = 0.305)

            # Color image canvas
            self.color_canvas = Canvas(self.master, bg = 'gray25')
            self.color_canvas.place(relx = 0, rely = 0, relwidth = 0.3515, relheight = 0.625)  # change to ratio, this works for 1440x900 resolution 
            
            # ST image canvas
            self.st_canvas = Canvas(self.master, bg = 'gray25')
            self.st_canvas.place(relx = 0.3515, rely = 0, relwidth = 0.3515, relheight = 0.625)
            
            # frame loading canvas
            self.frame_loading_canvas = Canvas(self.master, bg = 'honeydew2')
            self.frame_loading_canvas.place(relx = 0.703, rely = 0, relwidth = 0.297, relheight = 0.625)
        
        if self.detect_width == 1536 and self.detect_height == 864: #580
            self.video_feed_dim = 580
            # Middle canvas
            self.frame_selector_canvas = Canvas(self.master, bg = 'honeydew2')
            self.frame_selector_canvas.place(x = 0, rely = 0.6365, relwidth = 1, relheight = 0.07)
            
            # Ethogram canvas
            self.etho_canvas = Canvas(self.master, bg = 'bisque', bd = 0) 
            self.etho_canvas.place(relx = 0, rely = 0.7095, relwidth = 1, relheight = 0.2905)

            # Color image canvas
            self.color_canvas = Canvas(self.master, bg = 'gray25')
            self.color_canvas.place(relx = 0, rely = 0, relwidth = 0.358, relheight = 0.6365)  # change to ratio, this works for 1440x900 resolution 
            
            # ST image canvas
            self.st_canvas = Canvas(self.master, bg = 'gray25')
            self.st_canvas.place(relx = 0.358, rely = 0, relwidth = 0.358, relheight = 0.6365)
            
            # frame loading canvas
            self.frame_loading_canvas = Canvas(self.master, bg = 'honeydew2')
            self.frame_loading_canvas.place(relx = 0.716, rely = 0, relwidth = 0.284, relheight = 0.6365)
    
        self.scaler = Scale(self.frame_selector_canvas, from_ = 0, to = self.ethoLength, orient = 'horizontal',
                           tickinterval = self.ethoLength)
        self.scaler.place(relx = 0.007, rely = 0.1, relwidth = 0.743, relheight = 0.8)
        
        
        
        self.backward = Button(self.frame_selector_canvas, text = "<", command = self.move_frame_backward)
        self.backward.place(relx = 0.76, rely = 0.15, relwidth = 0.02, relheight = 0.7)
        
        self.forward = Button(self.frame_selector_canvas, text = '>', command = self.move_frame_forward)
        self.forward.place(relx = 0.78, rely = 0.15, relwidth = 0.02, relheight = 0.7)
        
        self.zoomPlus = Button(self.frame_selector_canvas, text = '(+)', command = self.zoom_in)
        self.zoomPlus.place(relx = 0.815, rely = 0.05, relwidth = 0.03, relheight = 0.45)
        
        self.zoomMinus = Button(self.frame_selector_canvas, text = '(-)', command = self.zoom_out)
        self.zoomMinus.place(relx = 0.855, rely = 0.05, relwidth = 0.03, relheight = 0.45)
        
        self.zoomXLabel = Label(self.frame_selector_canvas, text = "Zoom 0x", bg = 'honeydew2')
        self.zoomXLabel.place(relx = 0.81, rely = 0.5, relwidth = 0.08, relheight = 0.45)
        
        self.entry_correct = Entry(self.frame_selector_canvas)
        self.entry_correct.place(relx = 0.91, rely = 0.15, relwidth = 0.03, relheight = 0.7)
        
        self.Correct_button = Button(self.frame_selector_canvas, text = "Correct", command = self.correct_ethogram)
        self.Correct_button.place(relx = 0.94, rely = 0.15, relwidth = 0.05, relheight = 0.7)
        
        
        self.labelTextBox = Label(self.st_canvas)
        self.labelTextBox.place(relx=0.8, rely=0.95, relwidth = 0.2, relheight = 0.05)

        self.entryStartFrame = Entry(self.frame_loading_canvas)
        self.entryStartFrame.place(relx = 0.05, rely = 0.35, relwidth = 0.2, relheight = 0.07)

        self.labelTo = Label(self.frame_loading_canvas, text = "to", bg = "honeydew2")
        self.labelTo.place(relx = 0.26, rely = 0.35, relwidth = 0.08, relheight = 0.07)

        self.entryEndFrame = Entry(self.frame_loading_canvas)
        self.entryEndFrame.place(relx = 0.35, rely = 0.35, relwidth = 0.2, relheight = 0.07)
        
        
        self.InfoCanvas = Canvas(self.frame_loading_canvas, bd = 3)
        self.InfoCanvas.place(relx = 0.05, rely = 0.02, relwidth = 0.9, relheight = 0.28)              
        
        self.scaler = Scale(self.frame_selector_canvas, from_ = 0, to = self.ethoLength, orient = 'horizontal',
                           tickinterval = self.ethoLength)
        self.scaler.place(relx = 0.007, rely = 0.1, relwidth = 0.743, relheight = 0.8)
        
        
        self.backward["state"] = "disabled"
        self.forward["state"] = "disabled"
        self.zoomPlus["state"] = "disabled"
        self.zoomMinus["state"] = "disabled"
        self.entry_correct["state"] = "disabled" 
        self.Correct_button["state"] = "disabled" 
        self.scaler["state"] = "disabled"

        self.diff_lb = Listbox(self.InfoCanvas)
        self.diff_lb.place(relx = 0, rely = 0, relwidth = 0.33, relheight = 1)
        
        self.new_lb = Listbox(self.InfoCanvas)
        self.new_lb.place(relx = 0.33, rely = 0, relwidth = 0.67, relheight = 1)
        
        self.calculate_video_info()
        
        self.diff_lb.insert(END, 'Video File')
        self.diff_lb.insert(END, 'Frame Count')
        self.diff_lb.insert(END, 'Duration')
        self.diff_lb.insert(END, 'FPS')
        self.diff_lb.insert(END, 'Quadrant')
        self.diff_lb.insert(END, 'Model File')
        
        
        self.new_lb.insert(END, str(self.video_source))
        self.new_lb.insert(END, str(self.frame_count))
        self.new_lb.insert(END, str(self.duration) +' sec')
        self.new_lb.insert(END, str(self.fps))
        self.new_lb.insert(END, str(self.quadrant_name))
        self.new_lb.insert(END, str(self.ABRS_model_path))
       
        self.HistoryCanvas = Canvas(self.frame_loading_canvas, bd = 3)
        self.HistoryCanvas.place(relx = 0.05, rely = 0.54, relwidth = 0.9, relheight = 0.44)
        
        self.HistoryCanvasfirstFrame = Frame(self.HistoryCanvas)
        self.HistoryCanvasfirstFrame.pack(fill=BOTH,expand=1)
        
        self.HistoryCanvas2 = Canvas(self.HistoryCanvasfirstFrame)
        self.HistoryCanvas2.pack(side=LEFT,fill=BOTH,expand=1)
        
        self.History_scrollbar = ttk.Scrollbar(self.HistoryCanvasfirstFrame, orient = VERTICAL, command = self.HistoryCanvas2.yview)
        self.History_scrollbar.pack(side=RIGHT,fill=Y)

        self.HistoryCanvas2.configure(yscrollcommand = self.History_scrollbar.set)
        self.HistoryCanvas2.bind('<Configure>', lambda e: self.HistoryCanvas2.configure(scrollregion = self.HistoryCanvas2.bbox('all')))
        
        self.Historysecond_frame = Frame(self.HistoryCanvas2)
        self.HistoryCanvas2.create_window((0,0), window = self.Historysecond_frame, anchor = 'nw')
        
        self.HistoryLabel = Label(self.Historysecond_frame, text = "Program Start at " + datetime.datetime.now().strftime(("%H:%M:%S %Y-%m-%d")), justify = LEFT)

        self.history_counter = 0
        self.HistoryLabel.grid(row = self.history_counter, column = 0, sticky = W)
        self.increase_history_counter()

        
        self.load = Button(self.frame_loading_canvas, text = "Load",
                           command = self.create_buffer_obj)  # this loads the inputs
                                # of start and end frames from entry widgets and creates the buffer object at the same time
        self.load.place(relx = 0.7, rely = 0.35, relwidth = 0.25, relheight = 0.07)        
        self.load_all_button = Button(self.frame_loading_canvas, text = "Load All Frames",
                           command = self.load_all_frames)  
        self.load_all_button.place(relx = 0.55, rely = 0.45, relwidth = 0.4, relheight = 0.07)
       
    def minimize_window(self):
        if Toplevel.winfo_exists(self.StartPage) == True:
            self.StartPage.iconify()
            self.master.attributes('-fullscreen', False)
            self.master.iconify()
        else:
            self.master.attributes('-fullscreen', False)
            self.master.iconify()
            
    def maximize_window(self):
        if Toplevel.winfo_exists(self.StartPage) == True:
            self.master.deiconify()
            self.master.attributes('-fullscreen', True)
            self.StartPage.deiconify()
            
        else:
            self.master.deiconify()
            self.master.attributes('-fullscreen', True)
        
    def save_labels_and_images(self):
        self.master.attributes('-topmost', False)
        response_save = messagebox.askyesno('', 'Do you wish to save labels and ST images and finish?', parent = self.master)
        
        if response_save == 1:
            STimRec = self.bufferObj.get_ST_image_record()
            frameNameStart = str(self.startFrame)
            frameNameEnd = str(self.endFrame)
            savedDummyData = etho_mat_2_etho_vect(self.NewDummyData)
                        
            dump_path = os.path.join(os.getcwd(), self.project_name)
            
            if not (os.path.isdir(dump_path)):
                os.mkdir(dump_path)
                
            with open(dump_path + '/' + 'ethoLabel' + str(self.video_source).rsplit('//', 1)[1] + "_fb" + str(self.quadrant) + "_" + frameNameStart + '_' + frameNameEnd, "wb") as f:
                pickle.dump(savedDummyData, f)
               
            with open(dump_path + '/' + 'STimRec' + str(self.video_source).rsplit('//', 1)[1]  + "_fb" + str(self.quadrant) + "_" + frameNameStart + '_' + frameNameEnd, "wb") as f:
                pickle.dump(STimRec, f)
            self.master.quit()
            self.master.destroy()
            
        else:
            pass 
            self.master.attributes('-topmost', True)

    def calculate_video_info(self):
        if self.first_time_bool == False:             
            self.vid = cv2.VideoCapture(self.video_source)
            self.frame_count = int(self.vid.get(cv2.CAP_PROP_FRAME_COUNT))
            self.fps = int(self.vid.get(cv2.CAP_PROP_FPS))
            self.duration = int(self.frame_count/self.fps)
        else:
            self.frame_count = None
            self.fps = None
            self.duration = None
    
    def check_customization(self):
        if self.project_name == "Untitled Project" or self.project_name == "":
            return False, "Please select a valid project name. Default unavailable."
        
        if self.video_source == "None":
            return False, "Please select a file for the video. Default unavailable."
        
        if self.ABRS_model_path == "None":
            return False, "Please select a file for the model. Default unavailable."
        
        if self.quadrant == 5:
            return False, "Please select a quadrant/whole frame. Default unavailable."
        
        if self.first_time_bool == True:    
            self.first_time_bool = False
            pickle.dump(self.dictParam, open("ABRS_cust.dat", "wb"))        
        return True, "All good"
        
    
    def load_all_frames(self):
        self.master.attributes('-topmost', False)
        response_load = messagebox.askyesno('', 'Are you sure you want to load all the frames in the video? This will take several minutes.', parent = self.master)        
        if response_load == 1:
            self.startFrame = 0
            self.endFrame = self.frame_count
            self.load_all_framesBool = True
            self.create_buffer_obj()
            self.load_all_framesBool = False  # you can also pass in 1 to 100
        else:
            pass
            self.master.attributes('-topmost', True)
    
    def increase_history_counter(self):
        self.history_counter = self.history_counter + 1


    def move_frame_forward(self): 
        if self.scaler.get() < self.endFrame:  # change to whatever ultimate length, done yes
            self.scaler_val = self.scaler.get() + 1
            self.scaler.set(self.scaler_val)
        else:
            pass
        self.boolBottomPressed = True
    
    def move_frame_backward(self):
        if self.scaler.get() > self.startFrame:
            self.scaler_val = self.scaler.get() - 1
            self.scaler.set(self.scaler_val)
        else:
            pass
        self.boolBottomPressed = True # it is false

    def call_attribute(self):
        self.master.attributes('-topmost', False)
    
    def zoom_in(self):            
        if self.ZoomHome == True:
            self.ZoomHome = False
            self.Zoom1x = True
            self.zoomMinus["state"] = "normal"
            if self.FirstSelectorBool2 == True and self.SecondSelectorBool2 == False:
                if (self.in_xi_FirstSelector) <= (int((self.endFrame - self.startFrame + 1)/2)):
                    self.in_Zoom1xFirstFrame = 0
                    self.in_Zoom1xSecondFrame = self.in_xi_FirstSelector  # we are not adding + here, we are going to add it when we call draw ethogram nad we index into the stuff
                    self.zoomIndStart = self.in_Zoom1xFirstFrame
                    self.zoomIndEnd = self.in_Zoom1xSecondFrame + 1
                else:
                    self.in_Zoom1xFirstFrame = self.in_xi_FirstSelector
                    self.in_Zoom1xSecondFrame = self.endFrame - self.startFrame
                    self.zoomIndStart = self.in_Zoom1xFirstFrame
                    self.zoomIndEnd = self.in_Zoom1xSecondFrame + 1

            if self.FirstSelectorBool2 == True and self.SecondSelectorBool2 == True:
                if self.in_xi_FirstSelector > self.in_xi_SecondSelector:
                    self.in_Zoom1xFirstFrame = self.in_xi_SecondSelector
                    self.in_Zoom1xSecondFrame = self.in_xi_FirstSelector
                    self.zoomIndStart = self.in_Zoom1xFirstFrame
                    self.zoomIndEnd = self.in_Zoom1xSecondFrame + 1
                    
                if self.in_xi_FirstSelector <= self.in_xi_SecondSelector:
                    if self.in_xi_FirstSelector < self.in_xi_SecondSelector:
                        self.in_Zoom1xFirstFrame = self.in_xi_FirstSelector
                        self.in_Zoom1xSecondFrame = self.in_xi_SecondSelector
                        self.zoomIndStart = self.in_Zoom1xFirstFrame
                        self.zoomIndEnd = self.in_Zoom1xSecondFrame + 1
                        
                    if self.in_xi_FirstSelector == self.in_xi_SecondSelector:
                        if (self.in_xi_FirstSelector <= (int((self.endFrame - self.startFrame + 1)/2))):
                            self.in_Zoom1xFirstFrame = 0
                            self.in_Zoom1xSecondFrame = self.in_xi_FirstSelector # we are not adding + here, we are going to add it when we call draw ethogram nad we index into the stuff
                            self.zoomIndStart = self.in_Zoom1xFirstFrame
                            self.zoomIndEnd = self.in_Zoom1xSecondFrame + 1
                        else:
                            self.in_Zoom1xFirstFrame = self.in_xi_FirstSelector
                            self.in_Zoom1xSecondFrame = self.endFrame - self.startFrame
                            self.zoomIndStart = self.in_Zoom1xFirstFrame
                            self.zoomIndEnd = self.in_Zoom1xSecondFrame + 1
            self.scaler.config(from_ = self.in_Zoom1xFirstFrame + self.startFrame, to = self.in_Zoom1xSecondFrame + self.startFrame, tickinterval = self.in_Zoom1xSecondFrame - self.in_Zoom1xFirstFrame)
            self.scaler.set(self.zoomIndStart + self.startFrame)
            self.zoomXLabel.configure(text = 'Zoom 1x')
            self.draw_ethogram()
    
    def zoom_out(self):
        self.Zoom1x = False
        self.ZoomHome = True
        self.zoomMinus["state"] = "disabled"
        self.zoomIndStart = 0
        self.zoomIndEnd = self.endFrame - self.startFrame + 1
        self.scaler.config(from_ = self.startFrame, to = self.endFrame, tickinterval = self.endFrame - self.startFrame)
        self.scaler.set(self.scalePosInd)
        self.zoomXLabel.configure(text = 'Zoom 0x')
        self.draw_ethogram()

    def helper_function_drawetho_historyUpdate(self, pass_in_text):
        self.HistoryLabel = Label(self.Historysecond_frame, text = pass_in_text, justify = LEFT, fg = 'red')
        self.HistoryLabel.grid(row = self.history_counter, column = 0, sticky = W)
        self.increase_history_counter()                           
        self.master.update_idletasks()
        self.HistoryCanvas2.configure(scrollregion=self.HistoryCanvas2.bbox("all")) 
        self.HistoryCanvas2.yview_moveto(1)

        
    def draw_ethogram(self):
        if self.draw_etho_first == True:
            self.NewDummyData = self.bufferObj.get_dummy_labels()
            self.New_labelRec = self.bufferObj.get_labelRec() - 1 # for actual labels we have to convert it back, remember to do this. !!??
            self.Secondary_labelRec = np.zeros(((self.endFrame - self.startFrame) + 1, 1))  # first fill with zeroes
            self.Secondary_labelRec.fill(np.nan) # now fill with nan
            self.ZoomHome = True
                
        if self.Zoom1x == True:
            self.in_xi_FirstSelector = self.in_xi_FirstSelector + self.zoomIndStart
            self.in_xi_SecondSelector = self.in_xi_SecondSelector + self.zoomIndStart
        if self.Zoom1x == False:
            pass
            
        # this is false when running correct function, here, the self.DummyData changes
        if self.draw_etho_first == False and self.correctingBool == True:
            if self.FirstSelectorBool2 == True and self.SecondSelectorBool2 == False:
                
                self.NewDummyData[int(self.New_labelRec[self.in_xi_FirstSelector]), self.in_xi_FirstSelector] = 0 # making old label 0 to reflect change
                self.NewDummyData[self.label_to_correct-1, self.in_xi_FirstSelector] = 1  # making new label 1, # i do -1 here for label to correct because python goes from 0-8 not 1-9
                self.OldLabel = int(self.New_labelRec[self.in_xi_FirstSelector]) + 1
                self.New_labelRec[self.in_xi_FirstSelector] = np.array([self.label_to_correct]) - 1
                
                newtext = datetime.datetime.now().strftime(("%H:%M:%S")) + ' At frame ' + str(self.in_xi_FirstSelector + self.startFrame) + ',\nlabel ' + str(self.OldLabel) + ' corrected to ' + str(self.label_to_correct)
                self.helper_function_drawetho_historyUpdate(newtext)
                
                if self.two_label_inputs == True:
                    if not (np.isnan(self.Secondary_labelRec[self.in_xi_FirstSelector])):
                        self.NewDummyData[int(self.Secondary_labelRec[self.in_xi_FirstSelector]), self.in_xi_FirstSelector] = 0
                    self.NewDummyData[self.label_to_correct2-1, self.in_xi_FirstSelector] = 0.6
                    self.Secondary_labelRec[self.in_xi_FirstSelector] = np.array([self.label_to_correct2]) - 1
                    
                    newtext = datetime.datetime.now().strftime(("%H:%M:%S")) + ' At frame ' + str(self.in_xi_FirstSelector + self.startFrame) + ',\nsecondary label ' + str(self.label_to_correct2) + ' added'
                    self.helper_function_drawetho_historyUpdate(newtext)
                    
                
                self.set_first_selector_line_back = True # this is for the if block to set the selector lines back
                self.set_both_selector_lines_back = False
                
            if self.FirstSelectorBool2 == True and self.SecondSelectorBool2 == True:
                if self.in_xi_FirstSelector > self.in_xi_SecondSelector:
                    
                    for frame_to_zero in range(self.in_xi_SecondSelector, self.in_xi_FirstSelector + 1):
                        self.NewDummyData[int(self.New_labelRec[frame_to_zero]), frame_to_zero] = 0
                        
                    self.NewDummyData[self.label_to_correct-1, self.in_xi_SecondSelector:self.in_xi_FirstSelector + 1] = 1  # the reason this is +1 bc python doesn't change the last value
                    self.New_labelRec[self.in_xi_SecondSelector:self.in_xi_FirstSelector + 1] = np.array([self.label_to_correct]) - 1
                    
                    newtext = datetime.datetime.now().strftime(("%H:%M:%S")) + ' At frames ' + str(self.in_xi_SecondSelector + self.startFrame) + ' to ' + str(self.in_xi_FirstSelector + self.startFrame) + ',\nall labels corrected to ' + str(self.label_to_correct)
                    self.helper_function_drawetho_historyUpdate(newtext)
                    
                    if self.two_label_inputs == True:
                        for new_frame_to_zero in range(self.in_xi_SecondSelector, self.in_xi_FirstSelector + 1):
                            if not (np.isnan(self.Secondary_labelRec[new_frame_to_zero])):
                                self.NewDummyData[int(self.Secondary_labelRec[new_frame_to_zero]), new_frame_to_zero] = 0
                        
                        self.NewDummyData[self.label_to_correct2-1, self.in_xi_SecondSelector:self.in_xi_FirstSelector + 1] = 0.6
                        self.Secondary_labelRec[self.in_xi_SecondSelector:self.in_xi_FirstSelector + 1] = np.array([self.label_to_correct2]) - 1 
                        
                        newtext = datetime.datetime.now().strftime(("%H:%M:%S")) + ' At frames ' + str(self.in_xi_SecondSelector + self.startFrame) + ' to ' + str(self.in_xi_FirstSelector + self.startFrame) + ',\nsecondary label ' + str(self.label_to_correct2) + ' added'
                        self.helper_function_drawetho_historyUpdate(newtext)
                
                
                if self.in_xi_FirstSelector <= self.in_xi_SecondSelector:
                     
                    for frame_to_zero in range(self.in_xi_FirstSelector, self.in_xi_SecondSelector + 1):
                        self.NewDummyData[int(self.New_labelRec[frame_to_zero]), frame_to_zero] = 0
                    
                    self.NewDummyData[self.label_to_correct-1, self.in_xi_FirstSelector:self.in_xi_SecondSelector + 1] = 1  # the reason this is +1 bc python doesn't change the last value
                    self.OldLabel = int(self.New_labelRec[self.in_xi_FirstSelector]) + 1
                    self.New_labelRec[self.in_xi_FirstSelector:self.in_xi_SecondSelector+1] = np.array([self.label_to_correct]) - 1
                    
                    if self.in_xi_FirstSelector < self.in_xi_SecondSelector:
                        newtext = datetime.datetime.now().strftime(("%H:%M:%S")) + ' At frames ' + str(self.in_xi_FirstSelector + self.startFrame) + ' to ' + str(self.in_xi_SecondSelector + self.startFrame) + ',\nall labels corrected to ' + str(self.label_to_correct)
                        self.helper_function_drawetho_historyUpdate(newtext)
                    
                    if self.in_xi_FirstSelector == self.in_xi_SecondSelector:
                        newtext = datetime.datetime.now().strftime(("%H:%M:%S")) + ' At frame ' + str(self.in_xi_FirstSelector + self.startFrame) + ', label ' + str(self.OldLabel) + ' corrected to ' + str(self.label_to_correct)
                        self.helper_function_drawetho_historyUpdate(newtext)
                    
                    if self.two_label_inputs == True:
                        for new_frame_to_zero in range(self.in_xi_FirstSelector, self.in_xi_SecondSelector + 1):
                            if not (np.isnan(self.Secondary_labelRec[new_frame_to_zero])):
                                self.NewDummyData[int(self.Secondary_labelRec[new_frame_to_zero]), new_frame_to_zero] = 0
                        
                        self.NewDummyData[self.label_to_correct2-1, self.in_xi_FirstSelector:self.in_xi_SecondSelector + 1] = 0.6
                        self.Secondary_labelRec[self.in_xi_FirstSelector:self.in_xi_SecondSelector+1] = np.array([self.label_to_correct2]) - 1
                        
                        newtext = datetime.datetime.now().strftime(("%H:%M:%S")) + ' At frames ' + str(self.in_xi_FirstSelector + self.startFrame) + ' to ' + str(self.in_xi_SecondSelector + self.startFrame) + ',\nsecondary label ' + str(self.label_to_correct2) + ' added'
                        self.helper_function_drawetho_historyUpdate(newtext)                                               
                self.set_both_selector_lines_back = True
                self.set_first_selector_line_back = False
                
            self.correctingBool = False   
        
        fig = Figure()
        self.ax = fig.add_subplot(111)
        fig.set_tight_layout(True)

        cax = self.ax.matshow(self.NewDummyData[:, self.zoomIndStart:self.zoomIndEnd], interpolation='nearest', aspect = 'auto')
        b = range(self.zoomIndStart + self.startFrame, self.zoomIndEnd + self.startFrame, int(self.ax.get_xticks()[1])-int(self.ax.get_xticks()[0]))
        a = list(map(str,b)) 

        self.ax.set_xticklabels([''] + a)
        self.ax.set_yticklabels([''] + self.alpha)
        self.Current_Frame_VSpan = self.ax.axvspan(xmin = 0-0.5, xmax = 0 + 0.5, ymin = 0.005, ymax = 0.99, ec ='red',
                                                   fill = False, lw = 1)

        self.blank = FigureCanvasTkAgg(fig, self.etho_canvas)
        self.blank.get_tk_widget().place(relx = 0, rely = 0, relwidth = 1, relheight = 1)
        self.mplcount = 0  # this is so that i can remove mutiple connections by 2
        
        if self.set_first_selector_line_back == True or self.set_both_selector_lines_back == True:
            self.mplcount = 4
        
        self.FirstSelectorBool = False
        self.SecondSelectorBool = False
        self.removeFirstandSecondBool = False
        self.FirstSelectorBool2 = False
        self.SecondSelectorBool2 = False
        self.togethernegativeoneBool = False
        self.entry_correct["state"] = "disabled" 
        self.Correct_button["state"] = "disabled"
        self.zoomPlus["state"] = "disabled"
        c1 = mplcursors.cursor(cax, multiple = True)
        @c1.connect("add")
        def _(sel):                
            # the purpose of this block, having self.mplcount = 4 is so that we can delete the previous selector lines on click, which happens inside sel function
            if self.mplcount == 4:
                self.delete_selector_lines()
                self.set_first_selector_line_back = False
                self.set_both_selector_lines_back = False 
                self.mplcount = -1 # the next click will just clear but still make a target bbox line after next if block
                self.togethernegativeoneBool = True  # this is so that when self.mplcount changes to 0 here, it doesn' priduce a bbox when clearing the setlinesback
            # this block removes the connections every 2 selecions
            self.mplcount = self.mplcount + 1
            if self.mplcount == 3:
                for s in c1.selections:
                    c1.remove_selection(s)
                self.mplcount = 0
                self.removeFirstandSecondBool = True
                self.entry_correct["state"] = "disabled" 
                self.Correct_button["state"] = "disabled"
                self.zoomPlus["state"] = "disabled"                
            if self.togethernegativeoneBool == True:
                for s in c1.selections:
                    c1.remove_selection(s)              
            if self.togethernegativeoneBool == False:
                sel.annotation.get_bbox_patch().set(fc="white", ec="w", lw=2)
            self.xi = sel.target[0]
            self.xi = self.round_half_up(self.xi, 0)
            self.xi = self.startFrame+int(self.xi)
            if self.togethernegativeoneBool == False:
                sel.annotation.set_text(f'{self.xi + self.zoomIndStart}')
            self.togethernegativeoneBool = False            
            if self.mplcount == 1:
                self.in_xi_FirstSelector = self.xi - self.startFrame 
                self.FirstSelectorBool = True
                self.entry_correct["state"] = "normal" 
                self.Correct_button["state"] = "normal"
                if self.Zoom1x != True:
                    self.zoomPlus["state"] = "normal"
            if self.mplcount == 2:    
                self.in_xi_SecondSelector = self.xi - self.startFrame
                self.SecondSelectorBool = True
    
        self.blank.draw()
    
    def switch_raw_frame_zoom(self, event): # have to pass in event bc binding double click
        if self.raw_frame_zoomout == True:
            self.raw_frame_zoomout = False
        else:
            self.raw_frame_zoomout = True

    def correct_ethogram(self):
        self.correctingBool = True
        if "," in self.entry_correct.get():
            if self.entry_correct.get().count(",") != 1:
                messagebox.showerror('', 'One comma necessary in entry to separate the primary from the secondary label. The format should be "[Label #1 to change],[Label #2 to change]". For example, typing in "2,8" and pressing "Correct" if the user wishes to correct the primary label to 2 and add a secondary label 8', parent = self.master)
                return
            self.temp_entry_zero = self.entry_correct.get().split(",")[0].strip()
            self.temp_entry_one = self.entry_correct.get().split(",")[1].strip()
            if self.temp_entry_zero.isdigit():
                self.label_to_correct = int(self.temp_entry_zero)
                if self.label_to_correct > len(self.alpha) or self.label_to_correct <1:
                    messagebox.showerror('', 'The primary label is not within the range of ethogram label options finalized on the Start Page.', parent = self.master)
                    return
            if not self.temp_entry_zero.isdigit():
                messagebox.showerror('', 'To correct the primary label and to add a secondary label, the entry has to be two, comma-separated label numbers within range. The format should be "[Label #1 to change],[Label #2 to change]". For example, typing in "2,8" and pressing "Correct" if the user wishes to correct the primary label to 2 and add a secondary label 8', parent = self.master)
                return
            if self.temp_entry_one.isdigit():
                self.label_to_correct2 = int(self.temp_entry_one)
                if self.label_to_correct2 > len(self.alpha) or self.label_to_correct2 <1:
                    messagebox.showerror('', 'The secondary label is not within the range of ethogram label options finalized on the Start Page.', parent = self.master)
                    return
            if not self.temp_entry_one.isdigit():
                messagebox.showerror('', 'To correct the primary label and to add a secondary label, the entry has to be two, comma-separated label numbers within range. The format should be "[Label #1 to change],[Label #2 to change]". For example, typing in "2,8" and pressing "Correct" if the user wishes to correct the primary label to 2 and add a secondary label 8', parent = self.master)
                return
            if self.label_to_correct == self.label_to_correct2:
                messagebox.showerror('', 'The primary label to correct and secondary label to add cannot be the same.')
                return
            self.two_label_inputs = True
        if "," not in self.entry_correct.get():
            if self.entry_correct.get().strip().isdigit():
                self.label_to_correct = int(self.entry_correct.get().strip())
                if self.label_to_correct > len(self.alpha) or self.label_to_correct <1:
                    messagebox.showerror('', 'The label is not within the range of ethogram label options finalized on the Start Page.', parent = self.master)
                    return
            else:
                messagebox.showerror('', 'To correct to ONE label, the entry should have the the appropriate label number finalized on the on the Start Page.', parent = self.master)
                return
            self.two_label_inputs = False 
            
        
        if self.draw_etho_first == True:
            self.draw_etho_first = False 
            
        self.draw_ethogram()
        
        self.set_selector_lines_back_bool = True    
        
    def delete_selector_lines(self):
        if self.set_first_selector_line_back == True:
            self.FirstSelector_VSpan.remove()
        if self.set_both_selector_lines_back == True:
            self.BothSelector_VSpan.remove()
            
    def cancel_job(self):
        self.master.after_cancel(self.after_id)
        self.boolDoRun = False # make this false after a job, because we will make it true once clicked LOAD frames
    
    # this is used in making ethogram mpl cursor label when clicked 
    def round_half_up(self, n, decimals=0):
        multiplier = 10 ** decimals
        return math.floor(n*multiplier + 0.5) / multiplier
    
    def create_buffer_obj(self):
        
        if self.load_all_framesBool != True:
            self.tempsf = self.entryStartFrame.get()
            self.tempef = self.entryEndFrame.get()

            if (not self.tempsf.isdigit()) or (not self.tempef.isdigit()):
                messagebox.showerror('', 'Invalid frame inputs to load. Has to be a positive number or 0.', parent = self.master)
                return
            if (int(self.tempef) <= int(self.tempsf)) or (int(self.tempef) > self.frame_count):
                messagebox.showerror('',
                    'Invalid frame inputs to load. End frame input has to be greater than start frame and less than or equal to the total frame count.', parent = self.master)
                return
            self.startFrame = int(self.entryStartFrame.get())
            self.endFrame = int(self.entryEndFrame.get()) 
        
        self.load["state"] = "disabled"
        self.load_all_button["state"] = "disabled"
        self.entryStartFrame["state"] = "disabled"
        self.entryEndFrame["state"] = "disabled"        
        self.Zoom1x = False
        self.ZoomHome = True
        self.zoomIndStart = 0
        self.zoomIndEnd = self.endFrame - self.startFrame + 1
        self.scaler.config(from_ = self.startFrame, to = self.endFrame, tickinterval = self.endFrame - self.startFrame)
        self.scaler.set(self.startFrame)
        
        self.boolDoRun = True
        self.draw_etho_first = True 
        self.boolBottomPressed = False
        
        self.ethoCurrent = np.zeros((10, self.ethoLength))
        self.master.attributes('-topmost', False)

        self.alpha_len = int(len(self.alpha))            
        self.bufferObj = BufferRecord(self.ethoLength, self.ABRS_model_path, self.video_source, self.startFrame, self.endFrame, self.quadrant, self.alpha_len)              
        self.HistoryLabel = Label(self.Historysecond_frame, text = datetime.datetime.now().strftime(("%H:%M:%S")) + ' Frames ' + str(self.startFrame) + ' to ' + str(self.endFrame) + ' loaded', fg = 'blue', justify = LEFT)
        self.HistoryLabel.grid(row = self.history_counter, column = 0, sticky = W)
        self.increase_history_counter()
        self.master.update_idletasks()
        self.HistoryCanvas2.configure(scrollregion=self.HistoryCanvas2.bbox("all"))
        self.HistoryCanvas2.yview_moveto(1)    
        self.FirstRunBool = False
        
        self.master.attributes('-topmost', True)
        
        # After it is called once, the update method will be automatically called every delay milliseconds
        self.delay = 15

        self.draw_ethogram()
        
        self.backward["state"] = "normal"
        self.forward["state"] = "normal" 
        self.scaler["state"] = "normal"
        
        self.update() # calling this update function once, will let after function call it alot until job is cancelled.      
    
    def update(self):
        if self.boolBottomPressed == False:
            if self.scaler.get() < (self.endFrame + 1):  # Here, frameInd goes into relative mode, to indrelative              
                 self.scalePosInd = self.scaler.get()  
        noError = True

        if self.boolDoRun == True:  # this is true from when the create_bufferobj           
            self.ethoLength = self.dictParam['zoomEthoLength']            
            if self.raw_frame_zoomout == True:
                rawFrame = self.bufferObj.get_raw_frame(self.scalePosInd - self.startFrame) # it's easy to make a mistake here with the frame indicator
                rawFrame = cv2.resize(rawFrame, (self.video_feed_dim, self.video_feed_dim)) 
            elif self.raw_frame_zoomout == False:
                rawFrame =  self.bufferObj.get_ROI_raw_frame(self.scalePosInd - self.startFrame)
                rawFrame =  cv2.resize(rawFrame, (self.video_feed_dim, self.video_feed_dim))                        
            imST = self.bufferObj.get_ST_image(self.scalePosInd - self.startFrame)
            imST = cv2.resize(imST, (self.video_feed_dim, self.video_feed_dim))
            if noError == True:
                self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(rawFrame))  # makes image out of numpy matrix
                self.color_canvas.create_image(0, 0, image = self.photo, anchor = NW, tags = 'rawFrameTag')               
                
                if self.call_self_photo_bind == True:
                    self.call_self_photo_bind = False
                    self.color_canvas.tag_bind('rawFrameTag', '<Double-Button-1>', self.switch_raw_frame_zoom)                    
                self.imSTtoShow = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(imST))
                self.st_canvas.create_image(0, 0, image = self.imSTtoShow, anchor = NW)                
                self.labelTextBox['text'] = str(self.alpha[int(self.New_labelRec[self.scalePosInd - self.startFrame])])
                self.Current_Frame_VSpan.remove()                
                self.Current_Frame_VSpan = self.ax.axvspan(xmin = self.scalePosInd-self.startFrame - self.zoomIndStart -0.5, xmax = self.scalePosInd-self.startFrame - self.zoomIndStart + 0.5, ymin = 0.005, ymax = 0.99, ec ='red', fill = False, lw = 1)                
                
                if self.FirstSelectorBool == True:                    
                    self.SecondSelectorBool2 = False # Similar to self.cancel. We want to do this so that we can avoid the blue line that comes after we select white, correct, then selecte white and blue, then only white then correct               
                    self.FirstSelector_VSpan = self.ax.axvspan(xmin = self.in_xi_FirstSelector-0.5,
                                                           xmax = self.in_xi_FirstSelector+0.5, ymin = 0.005,
                                                           ymax = 0.99, ec ='lightpink',ls = '--', fill = False, lw = 2)                   
                    self.FirstSelectorBool = False
                    self.FirstSelectorBool2 = True  # bool2 is so that we can correct the position
                    
                if self.SecondSelectorBool == True:
                    self.FirstSelector_VSpan.remove() ## remove if going back
                    if self.in_xi_FirstSelector > self.in_xi_SecondSelector:
                        self.BothSelector_VSpan = self.ax.axvspan(xmin = self.in_xi_SecondSelector-0.5,
                                                                  xmax = self.in_xi_FirstSelector+0.5, ymin = 0.005, ymax = 0.99,
                                                                  ec ='lightpink', fill = False,ls = '--', lw = 2)
                    if self.in_xi_FirstSelector <= self.in_xi_SecondSelector:
                        self.BothSelector_VSpan = self.ax.axvspan(xmin = self.in_xi_FirstSelector-0.5,
                                                                  xmax = self.in_xi_SecondSelector+0.5, ymin = 0.005, ymax = 0.99,
                                                                  ec ='lightpink', fill = False,ls = '--', lw = 2)
                    self.SecondSelectorBool = False
                    self.SecondSelectorBool2 = True
                    
                if self.removeFirstandSecondBool == True:
                    self.BothSelector_VSpan.remove()
                    self.removeFirstandSecondBool = False
                
                if self.set_selector_lines_back_bool == True:
                    if self.set_first_selector_line_back == True:
                        self.FirstSelector_VSpan = self.ax.axvspan(xmin = self.in_xi_FirstSelector-self.zoomIndStart-0.5,
                                                           xmax = self.in_xi_FirstSelector-self.zoomIndStart+0.5, ymin = 0.005,
                                                           ymax = 0.99, ec ='lightpink',ls = '--', fill = False, lw = 2)
                        self.set_selector_lines_back_bool = False
                    if self.set_both_selector_lines_back == True:
                        if self.in_xi_FirstSelector > self.in_xi_SecondSelector:
                            self.BothSelector_VSpan = self.ax.axvspan(xmin = self.in_xi_SecondSelector-self.zoomIndStart-0.5,
                                                                  xmax = self.in_xi_FirstSelector-self.zoomIndStart+0.5, ymin = 0.005, ymax = 0.99,
                                                                  ec ='lightpink', fill = False,ls = '--', lw = 2)
                        if self.in_xi_FirstSelector <= self.in_xi_SecondSelector:
                            self.BothSelector_VSpan = self.ax.axvspan(xmin = self.in_xi_FirstSelector-self.zoomIndStart-0.5,
                                                                  xmax = self.in_xi_SecondSelector-self.zoomIndStart+0.5, ymin = 0.005, ymax = 0.99,
                                                                  ec ='lightpink', fill = False,ls = '--', lw = 2)
                        self.set_selector_lines_back_bool = False
                        
                self.blank.draw()
                self.boolBottomPressed = False
                
        self.after_id = self.master.after(self.delay, self.update)  # Self.Update function is called every 15 millisec
    
    
class BufferRecord:
    
    def __init__ (self, ethoLength, ABRS_model_path, video_source, startFrame, endFrame, quadrant, alpha_len): # video_source = 0                
        self.cap = cv2.VideoCapture(video_source)  
        if not self.cap.isOpened():
            raise ValueError("Unable to open video source", video_source)
        
        self.ABRS_model_path = ABRS_model_path
        self.alpha_len = alpha_len
        # Get cap source width and height
        self.frameWidth = self.cap.get(cv2.CAP_PROP_FRAME_WIDTH)
        self.frameHeight = self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
        self.new1024 = int(self.frameHeight/2)
        self.quadrant = quadrant 
        self.frameRecord, self.ROI_raw_frames, self.STRec, self.labelRec, self.probRec, self.DummyData = self.run_ABRS_in_window(ethoLength, startFrame, endFrame, self.quadrant)
        
    def get_raw_frame(self, ind):
        frame = self.frameRecord[ind,:,:]
        return frame
    
    def get_ROI_raw_frame(self, ind):
        ROI_frame = self.ROI_raw_frames[ind,:,:]        
        return ROI_frame

    def get_dummy_labels(self):
        return self.DummyData
    
    
    def get_ST_image(self, ind):
        im3CRaw = self.STRec[ind,:,:,:]
        
        if np.count_nonzero(im3CRaw[:,:,0])>6400:            
            im3CRaw[:,:,0] = np.zeros((80,80))
        
        if np.count_nonzero(im3CRaw[:,:,1])>800:            
            im3CRaw[:,:,1] = np.zeros((80,80))
        
        rgbArray = np.zeros((80,80,3), 'uint8') 
        rgbArray[..., 2] = im3CRaw[:,:,0]
        rgbArray[..., 1] = im3CRaw[:,:,1]
        rgbArray[..., 0] = im3CRaw[:,:,2]
        
        return rgbArray
    
    def get_labelRec(self):
        labels = self.labelRec
        return labels
    
    def get_probRec(self):
        probs = self.probRec
        return probs

    def get_ST_image_record(self):       
        return self.STRec 

    
    def run_ABRS_in_window(self, ethoLength, startFrame, endFrame, fb = 0):  # fb denotes quadrant or whole picture        
        self.ProgressPopUp = Toplevel()  # Top object
        self.ProgressPopUp.attributes('-topmost',True)
        self.ProgressPopUp.title('Loading frames...')
        self.ProgressPopUp.geometry('300x100')
        self.ProgressPopUp.resizable(False, False)
                
        ind = 0
        v = StringVar()
        v.set(str(ind) + '%')
        self.ProgressBarLabel = Label(self.ProgressPopUp, textvariable = v)
        self.ProgressBarLabel.grid(row=0,column=0)
        
        root.eval(f'tk::PlaceWindow {str(self.ProgressPopUp)} center')
        kernelSize = (endFrame - startFrame) + 1
        self.progress_bar = ttk.Progressbar(self.ProgressPopUp, maximum=(endFrame-startFrame))
        self.progress_bar.grid(row=1, column=0, ipadx = 100)
        
        newSize = (400,400)
        
        if fb>0:
            splitRatio = 2
        if fb==0:
            splitRatio = 1

        winST = 16  # this is related to the 16+1 in frRec
        beh = 0
        indBehDur = 0

        prevFrame = np.zeros((400,400))
        frameRecord = np.zeros((16+1, newSize[0]*newSize[1]))  # 160000 in the right slot

        frameCroppedRecord = np.zeros((kernelSize, int(self.frameHeight/splitRatio), int(self.frameWidth/splitRatio)))  # the splitRatio is adjusted for half of both
                                            # x and y, depends on which quadrant you want to do, if none, nothing is cut
                                            # if fb = 1, it will half of the frame's height but below that is specified
                                            # this np zeroes is made up of three dimensions
                                            # kernel Size is the number of matrices and the other two are ncol and nrow        
        self.conv_video_res = int(self.new1024*400/1024)
            
        if self.conv_video_res % 2 != 0 and splitRatio == 2:
            self.conv_video_res = int(self.new1024*400/1024) + 1
            
        self.half_conv_video_res = int(self.conv_video_res/splitRatio)
                    
        ROI_raw_frames = np.zeros((kernelSize, self.conv_video_res, self.conv_video_res)) 
        
        Master_gray_frames = np.zeros((self.new1024 + self.conv_video_res, self.new1024 + self.conv_video_res))
        
        STRec = np.zeros((kernelSize, 80, 80, 3))  # this is the ST image record with 80 columns and 80 rows for all frames
                # the 3 means it is in color
        labelRec = np.zeros((kernelSize, 1)) 
        probRec = np.zeros((kernelSize, 10))        
        DummyData = np.zeros((self.alpha_len, kernelSize)) 
    
        # Load model
        model = keras.models.load_model(self.ABRS_model_path)
        model.summary()
            
        for frameInd in range(startFrame - 1, endFrame, 1): 
            self.cap.set(cv2.CAP_PROP_POS_FRAMES, frameInd)  # Setting the position of the frame being read to
                # startFrame first frame, whatever startFrame is controls this loop
            ret, frame = self.cap.read() # Returns a bool (True if frame is read correctly) and the frame that will
                    # be read first is 1 because cap.read() reads the next frame and 0 is the current frame
            print(int(self.cap.get(cv2.CAP_PROP_POS_FRAMES)))  # print the current frame being read            
            if np.size(np.shape(frame)) != 0:            
                gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # converting from color photo to gray
            else:  # this will occur if frame is None type, which happens if frame doesn't get read
                        # or if nothing get's passed into video source and it stays 0
                gray_frame = np.zeros((int(self.frameHeight/splitRatio), int(self.frameHeight/splitRatio)))

            newSized_frame = cv2.resize(gray_frame, (newSize[0], newSize[1]))  #changing to 400 x 400

            croppedSize = int(np.shape(gray_frame)[0]/2)  # [0] is y, cutting it in half, the halfway mark

            if fb == 1:
                gray_frame_cropped = gray_frame[0:croppedSize, 0:croppedSize] # upper left quadrant of the video image
            if fb == 2:
                gray_frame_cropped = gray_frame[0:croppedSize, croppedSize:croppedSize*2]  # upper right 
            if fb == 3:
                gray_frame_cropped = gray_frame[croppedSize:croppedSize*2, 0:croppedSize]  # lower left 
            if fb == 4:
                gray_frame_cropped = gray_frame[croppedSize:croppedSize*2, croppedSize:croppedSize*2]  # lower right 
            if fb == 0:
                 gray_frame_cropped = gray_frame

            frameCroppedRecord[ind,:,:] = gray_frame_cropped  # indexing in to the first matrix of the array and putting
                    # in the current frame cropped to the specific size (half if fb > 0)
                # 1024 dimension

            currentFrame = newSized_frame.astype(float)/1   # this uses low res 400x400 frame

            diffFrame = currentFrame - prevFrame  # in the first case the prevFrame is np.zeros
            prevFrame = currentFrame  # this is where the current frame now becomes prev frame for the next run of the loop

            diffFrameAbs = np.absolute(diffFrame)  # absolute value of the frame float types

            frameVect = currentFrame.reshape(1, newSize[0]*newSize[1])  # putting the array into a vector, elements 
                    # are conserved
            frameVectFloat = frameVect.astype(float)  # making sure that it is of float type
            

            frameRecordShort = np.delete(frameRecord, 0, 0) # deleting the first row of frameRecord the +1
            frameRecord = np.vstack((frameRecordShort,frameVectFloat))  # vertically stacking the both arrays
                    # low res 3d stack

            sumframeRecord = np.sum(frameRecord,0)

            posDic, maxMovement, cfrVectRec, frameVectFloatRec = getting_frame_record(frameRecord, 0, winST, fb)  # this
                # is a function of the ABRS module, the cfrVectRec that is saved is used below
                #  unpack this dictionary and see where the fly is
            
            converted_x_pos = int(posDic["xPos"] * (self.new1024/200)) + self.half_conv_video_res
            converted_y_pos = int(posDic["yPos"] * (self.new1024/200)) + self.half_conv_video_res
            
            Master_gray_frames[self.half_conv_video_res:(self.new1024+self.half_conv_video_res),self.half_conv_video_res:(self.new1024+self.half_conv_video_res)] = gray_frame_cropped
            ROI_raw_frames[ind,:,:] = Master_gray_frames[converted_y_pos-self.half_conv_video_res: converted_y_pos+self.half_conv_video_res,
                                                             converted_x_pos-self.half_conv_video_res:converted_x_pos+self.half_conv_video_res]            
            
            im3CRaw = create_3C_image(cfrVectRec) # also a function of ABRS
            STRec[ind,:,:,:] = im3CRaw  # this is the St image self.STRec that will be called in the other function

            if np.count_nonzero(im3CRaw[:, :, 0]) > 6400:            
                im3CRaw[:, :, 0] = np.zeros((80, 80))

            if np.count_nonzero(im3CRaw[:, :, 1]) > 800:            
                im3CRaw[:, :, 1] = np.zeros((80, 80))

            rgbArray = np.zeros((80, 80, 3), 'uint8')
            rgbArray[..., 2] = im3CRaw[:, :, 0]
            rgbArray[..., 1] = im3CRaw[:, :, 1]
            rgbArray[..., 0] = im3CRaw[:, :, 2]
            
            X_newSized = np.zeros((1, 80, 80, 3))
        
            X_newSized[0, :, :, :] = im3CRaw

            X = X_newSized/1  # normalize

            predictionsProb = model.predict(X)

            predictionLabel = np.zeros((1, np.shape(predictionsProb)[0]))
            predictionLabel[0, :] = np.argmax(predictionsProb, axis=1)

            labelRec[ind] = predictionLabel
            
            gen = int(predictionLabel) - 1 # we do minus 1 so that 6 is 5 in etho but 6 in the code
            DummyData[gen, ind] = 1
            
            probRec[ind, :] = predictionsProb
            
            ind = ind + 1
            self.progress_bar['value'] = float(ind)
            self.ProgressPopUp.update()
            v.set(str(int(100*(ind/kernelSize))) + '%')
        self.ProgressPopUp.destroy()
        return frameCroppedRecord, ROI_raw_frames, STRec, labelRec, probRec, DummyData            
            
alpha = ['[1] Frontleg', '[2] Head', '[3] Abdominal', '[4] Backleg', '[5] Wing',
         '[6] Walking', '[7] Thorax', '[8] <empty>', '[9] Unknown']

if not (os.path.isfile('ABRS_cust.dat')):
    dictParam = {"zoomEthoLength" : 100, "ABRS_model_path" : None, "video_source" : None, 
             "quadrant" : 5, "project_name" : "Untitled Project", "alpha" : alpha, "first_time_bool" : True}
    pickle.dump(dictParam, open("ABRS_cust.dat", "wb"))
else:
    dictParam = pickle.load(open("ABRS_cust.dat", "rb"))

LabelMakerGUI(root, dictParam)
##################
#### END CODE ####
##################

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 86, 86, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 40, 40, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 40, 40, 64)   256         ['conv1_conv[0][0]']             
                                                                                              

In [9]:
#!pip install tk
#!pip install tensorflow
#!pip install matplotlib
#!pip install opencv-python
#!pip install --upgrade Pillow
#!pip install mplcursors
#!pip install scipy
#!pip install pandas
#!pip install natsort
#!pip install -U scikit-image
#!pip install -U scikit-learn

!pip install --upgrade jupyterlab ipympl

  Using cached ipywidgets-7.6.5-py2.py3-none-any.whl (121 kB)
  Using cached jupyterlab_widgets-1.0.2-py3-none-any.whl (243 kB)
  Using cached widgetsnbextension-3.5.2-py2.py3-none-any.whl (1.6 MB)
